# Simulating Language, Lab 4, Iterated Bayesian Learning

This simulation features a replication of the Reali & Griffiths iterated learning model of the evolution of frequency distributions, and is built around the Bayesian model of frequency learning/regularisation from Lab 3.  This simulation allows you to explore the effects of learning bias on cultural evolution. But before we get onto the model itself, we need to talk about log probabilities.

In [ ]:
from math import log, exp

## Introduction to log probabilities

In the lectures I introduced Bayes’ Rule as a relationship between probabilities: the posterior is proportional to the product of the likelihood and the prior, and all three of these quantities are probabilities. Doing Bayesian models of learning therefore involves manipulating probabilities, numbers between 0 and 1. And some of these probabilities can be very small indeed, because they involve multiplying small numbers lots of times (consider, for instance, how small the probability is of getting 100 heads if you flip a fair coin 100 times: it’s 0.5 x 0.5 x 0.5 ... 100 times, or $0.5^{100}$ if you prefer. That’s a very small number.)

As I said in the Lab 3 worksheet (go back and check if you need a refresher), working with small numbers on a computer can be a problem, because the computer cannot exactly represent real numbers (i.e. numbers we would write in decimal notation, e.g. numbers like 0.1, 3.147). There’s no way your computer can exactly represent every possible real number, so what it does instead is store numbers as accurately as it can, which involves introducing small rounding errors.  

Why do you need to care about this? Well, if you are dealing with very very small numbers (as you might do if you were doing a Bayesian model which involves learning from lots of data) then the rounding errors become a real factor - for big numbers the rounding errors are so small (relative to the big numers) we don’t really care, but for very small numbers, the rounding errors might be relatively big. Worse, sometimes the computer will round a very very small number to 0, which can generate unexpected and hard-to-predict errors in your code (e.g. if you try to divide something by a very very small number which gets rounded to 0).  

The solution to this is to have the computer work not with probabilities, but with *log probabilities*: we take our probabilities, take the log of those numbers, then carry on as before. We can convert log probabilities back to probabilities with the `exp` function, the inverse of the `log` function.

In [ ]:
print(log(1))
print(log(0.1))
print(log(0.000001))
print(exp(log(0.5)))
print(exp(log(0.1)))

As you can see from the code above, taking the log of a very small number turns it into a large negative number - these are still real numbers, so the computer still can’t represent them exactly, but in the log domain the rounding errors will be proportionately smaller for very small numbers and the rounding-to-0 problem won’t crop up. Then, if we want to see an actual probability, rather than a log probability, we can reverse this process, using the exp function, to get back raw probabilities. Jumping back and forth from logs can introduce rounding errors of its own, but it’s necessary to avoid the catastrophic rounding errors you can get if you just work with raw probabilities. 

Some basic arithmetic operations work a bit differently with logs. If you want to multiply two probabilities, you *add* their logarithms; if you want to divide one probability by another, you *subtract* the logarithm of one from another. And there is no direct equivalent of adding and subtracting in the log domain, which involves a little bit of fancy footwork in the code that you don’t have to worry about too much. The important thing is 1) to understand that the code is going to manipulate log probabilities and 2) this changes nothing conceptually, it’s just a matter of implementation.

In [ ]:
print(0.5 * 0.5)
print(exp(log(0.5) + log(0.5)))
print(0.5 / 0.5)
print(exp(log(0.5) - log(0.5)))

## On to the code

First, loading in the functions for random numbers and the beta distribution, plus some more that are specifically for doing stuff with logs. For instance, `logsumexp` (which we get from the `scipy.special` library allows us to do addition in the log domain (remember, just using the normal addition operator + with log probabilities is the equivalent of multiplying the raw probabilities). 

In [ ]:
import random
from scipy.stats import beta
from scipy.special import logsumexp
from math import log, log1p, exp

%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf')

The code starts with various bits and pieces which we need for working with logs and probability distributions. We define a function called `log_subtract` which allows us to do the equivalent of subtraction in the log domain (because if we just use normal subtraction, -, that’s equivalent to division). Then there are a couple of functions which we need for doing probabilistic sampling the log domain - `normalize_logprobs` will take a list of logs and normalise them for us (the equivalent of taking a list of pseudo-probabilities and rescaling them so they sum to 1, but in the log domain) and `log_roulette_wheel` takes a list of log probabilities and selects a random index from that list, with probability of any particular index being selected being given by its log probability. These are the log equivalents of `normalize_probs` and `roulette_wheel` from the Lab 3 code - they do exactly the same things, but with log probabilities rather than normal probabilities. As I said for the equivalent functions last week, *it is not important that you understand exactly how they work*. 

In [ ]:
def log_subtract(x,y):
    return x + log1p(-exp(y - x))

def normalize_logprobs(logprobs):
    logtotal = logsumexp(logprobs) #calculates the summed log probabilities
    normedlogs = []
    for logp in logprobs:
        normedlogs.append(logp - logtotal) #normalise - subtracting in the log domain
                                           #is equivalent to dividing in the normal domain
    return normedlogs

def log_roulette_wheel(normedlogs):
    r=log(random.random()) #generate a random number between 0 and 1, then convert to log
    accumulator = normedlogs[0] 
    for i in range(len(normedlogs)):
        if r < accumulator:
            return i
        accumulator = logsumexp([accumulator, normedlogs[i + 1]])

As in Lab 3, the main part of the code starts by laying out our hypothesis space, our grid of possible values of $\theta$ or `pW1`. We will need two grids here - one of normal probabilities (because they are easier to look at) and one of log probabilities (since we are going to work with log probabilities when we do our calculations). 

In [ ]:
grid_granularity = 100
grid_increment = 1 / grid_granularity
 
# sets up the grid of possible probabilities to consider
possible_pW1 = []
for i in range(grid_granularity):
    possible_pW1.append(grid_increment / 2 + (grid_increment * i))

# sets up the grid of log probabilities
possible_logpW1 = []
for pW1 in possible_pW1:
    possible_logpW1.append(log(pW1))

Have a look at the two grids (`possible_pW1` and `possible_logpW1`). Do they look like you expected?

Next up come the various functions we need for Bayesian inference. Again, these are all the same as last week, but just set up to work with log probabilities rather than normal probabilities. 

### The prior

In [ ]:
def calculate_logprior(alpha):
    logprior = []
    for pW1 in possible_pW1:
        logprior.append(beta.logpdf(pW1, alpha, alpha)) 
    return normalize_logprobs(logprior) 

This is exactly the same as the `calculate_prior` function from last week: you pass in an alpha parameter which determines the shape of the prior, it returns a list of values, one for each hypothesis in the hypothesis space, but the returned values are log probabilities rather than normal probabilities.   

- Plot some different prior probability distributions - for example, try typing `plt.plot(possible_pW1, calculate_logprior(0.1))` to see the prior log-probability distribution over various values of `pW1` for the `alpha=0.1` prior. It should be the same shape as you saw last week, but the values up the y axis will be different - they are now log probabilities rather than probabilities, but all you need to care about is the shape.

### Likelihood and production

As per the Lab 3 model, we are going to model data - sets of utterances - as a simple list of 0s and 1s: the 0s correspond to occurrences of word 0, the 1s correspond to occurrences of word 1. `loglikelihood` takes a log probability of word 1 being produced, and use that to calculate the probability of word 0 (which is 1 minus the probability of word 1). `logproduce` is the equivalent of `produce` last week, it generates some data based on a log-probability of producing word 1.

In [ ]:
def loglikelihood(data, logpW1):
    logpW0 = log_subtract(log(1), logpW1) #probability of w0 is 1-prob of w1
    logprobs = [logpW0, logpW1]
    loglikelihoods = []
    for d in data:
        loglikelihood_this_item = logprobs[d] #d will be either 0 or 1, 
                                              #so can use as index
        loglikelihoods.append(loglikelihood_this_item)
    return sum(loglikelihoods) #summing log probabilities = 
                               #multiply non-log probabilities
    
def logproduce(logpW1, n_productions):
    logpW0 = log_subtract(log(1), logpW1)
    logprobs = [logpW0, logpW1]
    data = []
    for p in range(n_productions):
        data.append(log_roulette_wheel(logprobs))
    return data

- Test out the `logproduce` function - remember, you need to feed it a log probability, so decide on a probability for w1 and then convert it to log using the log function. Check that it works as expected based on what you did last week. 
- Next, check out the `loglikelihood` function - how does the likelihood of a set of data depend on the data and the probability of word 1? It should work in exactly the same way as last week. Remember that the likelihood function returns a log probability, so you can convert this to a probability using the `exp` function if you want. 

### Learning

In [ ]:
def logposterior(data, logprior):
    posterior_logprobs = []
    for i in range(len(possible_logpW1)):
        logpW1 = possible_logpW1[i] 
        logp_h = logprior[i] #prior probability of this pW1
        logp_d = loglikelihood(data, logpW1) #likelihood of data given this pW1
        posterior_logprobs.append(logp_h + logp_d) #adding logs = 
                                                   #multiplying non-logs
    return normalize_logprobs(posterior_logprobs) 
     
def loglearn(data,prior):
    posterior_logprobs = logposterior(data, prior)
    selected_index = log_roulette_wheel(posterior_logprobs)
    return possible_logpW1[selected_index]

Now we have all the bits we need to calculate the posterior probability distribution, and therefore to do learning (by picking a hypothesis, a value of pW1, based on its posterior probability). You can verify that the posterior has the right sort of shape by doing something like this (which I adapted from the Lab 3 answer sheet): 
```python
my_prior = calculate_logprior(1)
my_data1 = [0] * 2 + [1] * 2
my_data2 = [0] * 4 + [1] * 4
my_data3 = [0] * 8 + [1] * 8
print('data1 = ',my_data1)
print('data2 = ',my_data2)
print('data3 = ',my_data3)

my_logposterior1 = logposterior(my_data1,my_prior)
my_logposterior2 = logposterior(my_data2,my_prior)
my_logposterior3 = logposterior(my_data3,my_prior)
plt.plot(possible_pW1, my_logposterior1,'grey',label='p(theta|my_data1)')
plt.plot(possible_pW1, my_logposterior2,'purple',label='p(theta|my_data2)')
plt.plot(possible_pW1, my_logposterior3,'red',label='p(theta|my_data3)')
plt.xlabel("theta")
plt.ylabel("logp(theta|data)")
plt.legend()
```

The numbers might look a bit different, because the y axis is logprobs, but the posterior should be highest where you expext it to be and low where you expect it to be low.

### Iterated learning

At last, we have all the bits we need to do iterated learning: we can have a learner infer a value of `pW1` given some observed data, using the `loglearn` function, then we can have that individual produce data which another individual can learn from, using the `logproduce` function.

In [ ]:
def iterate(alpha, n_productions, starting_count_w1, generations):
    prior = calculate_logprior(alpha)
    pW1_accumulator = [] #we build up lists of our inferred pW1 at each generation
    data_accumulator = [] #and number of produced word1 at each generation
    data = [1] * starting_count_w1 + [0] * (n_productions - starting_count_w1)
    for generation in range(generations):
        logpW1 = loglearn(data, prior)
        data = logproduce(logpW1, n_productions)
        pW1_accumulator.append(exp(logpW1)) #will convert the inferred pW1 to a normal probability for you!
        data_accumulator.append(sum(data))
    return pW1_accumulator, data_accumulator

You can run a simulation using something like:

```python
pW1_by_generation, data_by_generation = iterate(0.1, 10, 5, 20)
``` 

This will run the simulation for 20 generations, using a prior defined by alpha=0.1, each learner observes 10 data points before inferring pW1, and the initial language consists of 5 examples of word 1 (and therefore 5 of word 0). It returns two  values (this is the first time we've had a function do that, it's quote handy): a generation-by-generation record of the inferred values of pW1, and the data produced at each generation (specified as a number of occurences of word 1). It's worth plotting these values as a graph over time, but also looking at the histogram of the values to get a sense of how they are distributed overall. 

We can plot a simple line graph like this:
```python
pW1_by_generation, data_by_generation = iterate(0.1, 10, 5, 20)

plt.plot(pW1_by_generation)
plt.xlabel("generations")
plt.ylabel("theta")
```

And you can get a histogram of the `pW1` values across the entire simulation run like this:
```python
pW1_by_generation, data_by_generation = iterate(0.1, 10, 5, 1000)
plt.hist(pW1_by_generation)
plt.xlim(0, 1)
plt.xlabel("theta")
plt.ylabel("frequency")
```
The histograms might look better if you increase the number of generations *a lot*, like to 1000, to simulate a very long chain of transmission.

If you want to run a bunch of iterated learning chains, say 10, and plot them all on the same graph, you can do something like this:
```python
for i in range(10):
    pW1_by_generation, data_by_generation = iterate(0.1, 10, 5, 20)
    plt.plot(pW1_by_generation)
plt.xlabel("generations")
plt.ylabel("theta")
```

## Questions

Once you are happy that you understand that the log version of the code doesn't do anything different to the Lab 3 versions, and that you understand how iterated learning works, try these questions:

1. One of Reali & Griffiths’s main points was that studying learning in a single individual can be a bad way to discover their prior bias, particularly if you give them lots of data which swamps this prior bias - given enough data, learners with quite different priors look the same. Can you reproduce this effect using this code, or the Lab 3 code?
2. Iterated learning can potentially give a clearer picture of prior bias. Try running some simulations for 10 generations, with 10 data points passed from generation to generation, starting each simulation with 5 instances of w1 and 5 of w0. How does changing the prior change the results?  Try alpha=0.1, alpha=1, and alpha=5. Are the differences between different priors obvious after generation 1, or do they become more apparent over generations? Try running some very long chains (e.g. 1000 generations) with different priors and plotting histograms of the `pW1` values - can you see the effect of different priors here?
3. Now try messing with the amount of data that is passed from generation to generation (remember to change the starting count of the w1 so you can compare between the different data set sizes fairly). What happens if you pass more data between generations? What happens if you pass less? What happens if you pass no data from generation to generation? What would this latter setting correspond to in the real world?
4. If iterated learning is a model of language transmission in the real-world, what do these results imply about the kinds of languages we see in the wild, and how they relate to the prior biases of real language learners?   